In [1]:
from underworld import UWGeodynamics as GEO
from underworld import visualisation as vis

loaded rc file /home/jovyan/workspace/codes/UWGeodynamics_use_with_docker/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
u = GEO.UnitRegistry

In [3]:
# Characteristic values of the system
half_rate = (1.8 * u.centimeter / u.year).to(u.meter / u.second)
model_length = 60e3 * u.meter
model_height = 60e3 * u.meter
bodyforce = (3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2)

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"] = KM

In [4]:
Model = GEO.Model(elementRes=(16,16,16), 
                  minCoord=(-30 * u.kilometer, -30. * u.kilometer, -20 * u.kilometer),
                  maxCoord=( 30 * u.kilometer, 30. * u.kilometer,  10 * u.kilometer), 
                  gravity=(0., 0., -9.81 * u.meter / u.second**2))

In [5]:
Model.outputDir = "1_21_3D_ColumnsTractionBottom"

In [6]:
air = Model.add_material(name="air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=0.0))
background = Model.add_material(name="background", shape=GEO.shapes.Layer3D(top=air.bottom, bottom=Model.bottom))
heavyColumn = Model.add_material(name="Heavy Column", shape=GEO.shapes.Box(minX=-10.*u.kilometer,
                                                                           maxX=10. * u.kilometer,
                                                                           minY=-10. * u.kilometer,
                                                                           maxY=10. * u.kilometer,
                                                                           top=0, bottom=Model.bottom))

In [ ]:
Fig = vis.Figure(resolution=(1200,600))
Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.5)
Fig.window()

In [8]:
air.density =  0. * u.kilogram / u.metre**3
background.density = 2700. * u.kilogram / u.metre**3
heavyColumn.density = 3300. * u.kilogram / u.metre**3

In [9]:
import numpy as np
x = GEO.nd(-30 * u.kilometer)
y = GEO.nd(-30 * u.kilometer)
z = GEO.nd(-20 * u.kilometer)
P = Model.lithostatic_pressureField.evaluate(np.array([[x,y,z]]))
bottomPress = GEO.dimensionalise(P, u.megapascal)

In [10]:
air.viscosity         = 1e19 * u.pascal * u.second
background.viscosity  = 1e25 * u.pascal * u.second
heavyColumn.viscosity = 1e25 * u.pascal * u.second

In [11]:
Model.set_velocityBCs(left=[0.,0.,0.],
                      right=[0.,0.,0.],
                      front=[0., 0., 0.],
                      back=[0., 0., 0.])

Model.set_stressBCs(bottom=[0.,0., bottomPress])

In [12]:
Model.run_for(nstep=1)

Running with UWGeodynamics version 2.10.0-dev-71e6210(2.10-pre_release)
Options:  -Q22_pc_type uw -ksp_type bsscr -pc_type none -ksp_k2_type NULL -rescale_equations False -remove_constant_pressure_null_space False -change_backsolve False -change_A11rhspresolve False -restore_K False -A11_ksp_type fgmres -A11_ksp_rtol 1e-06 -scr_ksp_type fgmres -scr_ksp_rtol 1e-05
Step:     1 Model Time: 202.3 megayear dt: 202.3 megayear (2020-08-13 05:38:47)


1